# 实验一报告
文昱韦 2213125

#### 实验一：基于kNN 的手写数字识别
实验条件：给定semeion手写数字数据集，给定kNN分类算法
实验要求：
1. 初级要求：编程实现kNN算法；给出在不同k值（5，9，13）情况下，kNN算法对手写数字的识别精度（要求采用留一法）
2. 中级要求：与机器学习包或平台(如weka)中的kNN分类器结果进行对比，性能指标为精度ACC，其他指标如归一化互信息NMI、混淆熵CEN任选其一（或两者）
3. 高级要求：采用旋转等手段对原始数据进行处理，进行至少两个方向（左上，左下）旋转，采用CNN或其他深度学习方法实现手写体识别

首先导入相关工具包。

In [1]:
import numpy as np
from collections import Counter
from sklearn.model_selection import LeaveOneOut, train_test_split
from sklearn.metrics import normalized_mutual_info_score, confusion_matrix, accuracy_score
from sklearn.neighbors import KNeighborsClassifier
from scipy.ndimage import rotate
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from keras.utils import to_categorical

下面我们定义三个会用到的函数：

1. **euclidean_distance(a, b)**：计算两个向量$a$和$b$ 之间的欧几里得距离：
     $$
     d(a, b) = \sqrt{\sum_{i=1}^{n} (a_i - b_i)^2}
     $$
     
2. **calculate_nmi(labels, predictions)**：计算真实结果（标签）与预测结果之间的归一化互信息（NMI）。我们通过sklearn里的函数完成计算。

3. **calculate_cen(y_true, y_pred)**：计算混淆熵（CEN）。CEN 的公式为：
     $$
     CEN = -\sum_{i,j} P(i, j) \log(P(i, j))
     $$
   其中 $P(i, j)$是混淆矩阵的概率分布。

In [2]:
def euclidean_distance(a, b):
    return np.sqrt(np.sum((a - b) ** 2))
def calculate_nmi(labels, predictions):
    return normalized_mutual_info_score(labels, predictions)
def calculate_cen(y_true, y_pred):
    cm = confusion_matrix(y_true, y_pred)
    probs = cm / np.sum(cm)
    cen = -np.nansum(probs * np.log(probs + 1e-10))
    return cen

定义函数加载 Semeion 手写数字数据集，并进行预处理。**X**储存前 256 列表示图像的像素特征；将后 10 列one-hot 编码格式的标签的通过 `np.argmax` 转换为单一的数字标签（即0 到 9 之间的整数）储存在**y**中。

In [3]:
def load_semeion_data(file_path):
    data = np.loadtxt(file_path)
    # 前256列是特征，后10列是one-hot编码的标签
    X = data[:, :256]
    # 将one-hot编码转换为单一的数字标签
    y = np.argmax(data[:, 256:], axis=1)
    return X, y

下面是**初级要求**，即自己编程实现kNN，主要定义了两个函数。`knn_with_loocv`函数按照题目要求使用留一法评估分类器性能，并记录相关性能指标，它将训练集、测试集以及多个k值传入另一个函数`k_nearest_neighbors`，实现 kNN 分类。我们首先计算了测试样本与每个训练样本之间的距离并排序，然后对于每个给定的 $ k $ 值，我们选择距离最近的 $ k $ 个样本，并通过统计这 $ k $ 个样本中最常见的标签来进行预测。

In [4]:
def k_nearest_neighbors(train_data, train_labels, test_sample, k_values):
    distances = []
    # 计算测试样本与每个训练样本之间的距离并排序
    for i in range(len(train_data)):
        dist = euclidean_distance(train_data[i], test_sample)
        distances.append((dist, train_labels[i]))
    distances.sort(key=lambda x: x[0])
    predictions = [] # 储存该测试样本的三个预测值
    # 遍历 k_values 中的 k 值，获取每个 k 值对应的分类结果
    for k in k_values:
        k_neighbors = [distances[i][1] for i in range(k)]
        most_common = Counter(k_neighbors).most_common(1)
        predictions.append(most_common[0][0])
    return predictions

# 进行kNN分类，并使用留一法进行验证
def knn_with_loocv(data, labels, k_values):
    loo = LeaveOneOut()
    # 初始化一个字典，存储每个k值对应的预测结果
    predictions_dict = {k: {'correct':[], 'predictions':[], 'true_labels':[]} for k in k_values}
    # 留一法遍历数据
    for train_index, test_index in loo.split(data):
        train_data, test_data = data[train_index], data[test_index]
        train_labels, test_labels = labels[train_index], labels[test_index]
        predictions = k_nearest_neighbors(train_data, train_labels, test_data[0], k_values)
        
        # 将每个k值对应的预测存储到对应的列表中
        for i, k in enumerate(k_values):
            predictions_dict[k]['correct'].append(int(predictions[i] == test_labels[0]))
            predictions_dict[k]['predictions'].append(predictions[i])
            predictions_dict[k]['true_labels'].append(test_labels[0])
    # 计算每个k值的平均值
    accuracies = []
    for k in k_values:
        accuracy = np.mean(predictions_dict[k]['correct'])
        nmi = calculate_nmi(predictions_dict[k]['true_labels'], predictions_dict[k]['predictions'])
        cen = calculate_cen(predictions_dict[k]['true_labels'], predictions_dict[k]['predictions'])
        accuracies.append((accuracy, nmi, cen))
    return accuracies

下面**中级要求**，即使用 sklearn 的 kNN 分类器进行对比的部分，主要定义了一个函数 `compare_with_sklearn_knn`。这个函数遍历不同的$ k $ 值，利用 sklearn 的 `KNeighborsClassifier` 来评估模型性能。我们依旧使用留一法交叉验证，将数据集划分为训练集和测试集。在每次迭代中，我们拟合训练数据，并对测试样本进行预测，将预测结果和真实标签保存下来。之后，我们计算模型的准确率 (ACC)、归一化互信息 (NMI) 和混淆熵 (CEN)。最后，函数返回所有 $ k $ 值对应的准确率、NMI 和 CEN。这一部分为中级要求提供了性能对比，有助于验证自定义 kNN 实现的效果。

In [5]:
#################################
# sklearn
#################################
# 使用Scikit-learn的kNN分类器进行对比
def compare_with_sklearn_knn(X, y, k_values):
    accuracies = []
    nmis = []
    cens = []
    # 遍历不同的k值
    for k in k_values:
        # 使用Sklearn的KNeighborsClassifier
        knn = KNeighborsClassifier(n_neighbors=k)
        loo = LeaveOneOut()
        predictions = []
        true_labels = []
        for train_index, test_index in loo.split(X):
            train_data, test_data = X[train_index], X[test_index]
            train_labels, test_labels = y[train_index], y[test_index]
            # 拟合并预测
            knn.fit(train_data, train_labels)
            prediction = knn.predict(test_data)
            predictions.append(prediction[0])
            true_labels.append(test_labels[0])
        # 计算 ACC, NMI, CEN
        acc = accuracy_score(true_labels, predictions)
        nmi = calculate_nmi(true_labels, predictions)
        cen = calculate_cen(true_labels, predictions)
        accuracies.append(acc)
        nmis.append(nmi)
        cens.append(cen)
    return accuracies, nmis, cens

下面是**高级要求**关于图像旋转和 CNN 模型构建的部分，我们定义了两个主要函数。

`augment_data(X, y)`函数主要是对图像进行左上和左下方向的旋转。我们首先使用 `rotate` 函数将每个图像顺时针旋转 20 度和逆时针旋转 20 度，并将旋转后的图像展平，形成了一个新的数据集。

`build_cnn(input_shape, num_classes)`函数主要用于构建CNN模型。我们首先定义模型的输入形状和类别数。模型包括多个卷积层和池化层，以提取特征和降低维度。最后通过 `Flatten` 层将数据展平，并添加全连接层，输出类别预测。我们使用 `softmax` 激活函数以获取每个类别的概率，并采用 `categorical_crossentropy` 作为损失函数，优化器选用 Adam。函数返回构建好的模型，准备进行训练和评估。

In [6]:
#################################
# 旋转后使用CNN进行识别
#################################
# 数据增强：对图像进行左上、左下旋转
def augment_data(X, y):
    X_rotated_left_up = np.array([rotate(x.reshape(16, 16), angle=20, reshape=False).flatten() for x in X])
    X_rotated_left_down = np.array([rotate(x.reshape(16, 16), angle=-20, reshape=False).flatten() for x in X])
    
    # 将原始数据和增强数据拼接
    X_augmented = np.concatenate([X_rotated_left_up, X_rotated_left_down], axis=0)
    y_augmented = np.concatenate([y, y], axis=0)
    return X_augmented, y_augmented

# 构建CNN模型
def build_cnn(input_shape, num_classes):
    model = Sequential()
    model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=input_shape))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dense(num_classes, activation='softmax'))
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

下面就是实验的主要执行部分，我们分别实现了自定义 kNN 分类、与 Scikit-learn 的 kNN 进行对比，以及使用 CNN 进行分类。

对于**自行实现 kNN**，我们首先加载数据集，并设置不同的 k  值，然后调用 `knn_with_loocv` 函数，传入特征和标签，得到每个  k 值的准确率、NMI 和 CEN并输出。

对于**sklearn 实现 kNN**，我们调用 `compare_with_sklearn_knn` 函数，输出每个  k  值的准确率、NMI 和 CEN，我们可以直观地比较自定义 kNN 实现和 Skit-learn 实现之间的差异。

对于**使用CNN 进行分类**，我们通过 `augment_data` 函数对原始图像进行旋转，然后对数据进行了预处理，调整形状以适应 CNN 输入格式，并将标签进行了转换；调用 `build_cnn` 函数构建 CNN 模型在训练集上进行训练、在测试集上进行预测，输出模型的准确率、NMI 和 CEN。

In [7]:
print("初级要求：自行实现kNN")
# 加载数据
file_path = 'semeion.data'
X, y = load_semeion_data(file_path)
# 设定不同的k值
k_values = [5, 9, 13]
# 传入k值数组，得到每个k值的精度
results = knn_with_loocv(X, y, k_values)
# 输出每个k值对应的准确率
for i, k in enumerate(k_values):
    accuracy, nmi, cen = results[i]
    print(f'k={k} 时的精度: {accuracy:.4f}, NMI: {nmi:.4f}, CEN: {cen:.4f}')




初级要求：自行实现kNN
k=5 时的精度: 0.9140, NMI: 0.8372, CEN: 2.6725
k=9 时的精度: 0.9240, NMI: 0.8515, CEN: 2.6403
k=13 时的精度: 0.9153, NMI: 0.8377, CEN: 2.6711


In [8]:
print("中级要求：sklearn实现kNN")
# 调用对比函数并输出结果
sklearn_accuracies, sklearn_nmis, sklearn_cens = compare_with_sklearn_knn(X, y, k_values)
# 输出Sklearn kNN分类器的结果
for i, k in enumerate(k_values):
    print(f'k={k} 时的精度: {sklearn_accuracies[i]:.4f}, NMI: {sklearn_nmis[i]:.4f}, CEN: {sklearn_cens[i]:.4f}')

中级要求：sklearn实现kNN
k=5 时的精度: 0.9052, NMI: 0.8293, CEN: 2.6879
k=9 时的精度: 0.9115, NMI: 0.8336, CEN: 2.6796
k=13 时的精度: 0.9033, NMI: 0.8224, CEN: 2.7043


In [9]:

print("高级要求：图像旋转后使用CNN进行分类")
X_augmented, y_augmented = augment_data(X, y)
X_augmented = X_augmented.reshape(-1, 16, 16, 1)  # 将数据重塑为适合CNN输入的格式
y_augmented_categorical = to_categorical(y_augmented)  # 将标签转换为one-hot编码
# 划分训练集和测试集
X_train, X_test, y_train, y_test = train_test_split(X_augmented, y_augmented_categorical, test_size=0.1, random_state=42)
# 构建CNN模型
input_shape = (16, 16, 1)
num_classes = 10  # 手写数字类别数
model = build_cnn(input_shape, num_classes)
# 训练模型
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.1)
# 测试模型
y_test_labels = np.argmax(y_test, axis=1)  # 转换one-hot编码为标签
y_pred = np.argmax(model.predict(X_test), axis=1)  # 模型预测标签
# 计算准确率
accuracy = accuracy_score(y_test_labels, y_pred)
# 计算NMI和CEN
nmi = calculate_nmi(y_test_labels, y_pred)
cen = calculate_cen(y_test_labels, y_pred)
# 输出结果
print(f'准确率: {accuracy:.4f}')
print(f'NMI: {nmi:.4f}')
print(f'CEN: {cen:.4f}')

高级要求：图像旋转后使用CNN进行分类
Epoch 1/10
81/81 [==============================] - 1s 5ms/step - loss: 1.6683 - accuracy: 0.4407 - val_loss: 1.1354 - val_accuracy: 0.5958
Epoch 2/10
81/81 [==============================] - 0s 3ms/step - loss: 0.8770 - accuracy: 0.7295 - val_loss: 0.9532 - val_accuracy: 0.6864
Epoch 3/10
81/81 [==============================] - 0s 3ms/step - loss: 0.6397 - accuracy: 0.7992 - val_loss: 0.6421 - val_accuracy: 0.8014
Epoch 4/10
81/81 [==============================] - 0s 3ms/step - loss: 0.5168 - accuracy: 0.8457 - val_loss: 0.5298 - val_accuracy: 0.8293
Epoch 5/10
81/81 [==============================] - 0s 3ms/step - loss: 0.4031 - accuracy: 0.8814 - val_loss: 0.5063 - val_accuracy: 0.8397
Epoch 6/10
81/81 [==============================] - 0s 3ms/step - loss: 0.3418 - accuracy: 0.8934 - val_loss: 0.5035 - val_accuracy: 0.8293
Epoch 7/10
81/81 [==============================] - 0s 3ms/step - loss: 0.2675 - accuracy: 0.9159 - val_loss: 0.3735 - val_accuracy: 0.8606


#### 实验结果分析

1. **自定义 kNN 实现**：
   - 在自定义实现的 kNN 中，我们观察到对于 $ k = 9 $ 时，模型表现最佳，准确率达到 0.9240，NMI 为 0.8515，CEN 为 2.6403。这表明选择合适的 $ k $ 值可以显著提高分类性能。
   - 其他 $ k $ 值的表现也相对稳定，尤其是 $ k = 5 $ 和 $ k = 13 $，分别获得了 0.9140 和 0.9153 的准确率。这说明自定义的 kNN 实现有效地捕捉了数据特征。
   - 对比 Scikit-learn 的 kNN 实现，我们发现其在相同 $ k $ 值下的准确率均低于自定义实现，尤其在 $ k = 5 $ 和 $ k = 9 $ 时，差距明显，分别为 0.9052 和 0.9115。这表明自定义实现可能在特定数据集上更具优势。

2. **使用 Scikit-learn 的 kNN 实现**：
   - Scikit-learn 的 kNN 分类器在所有 $ k $ 值下的准确率、NMI 和 CEN 指标均低于自定义实现，显示出其在本实验中相对较弱的性能。

3. **图像旋转后使用 CNN 进行分类**：
   - 图像旋转后，我们构建了 CNN 模型并进行了训练。最终模型在测试集上获得准确率0.9248，NMI 为0.8658，CEN 为 2.5960。这表明即使图像进行了旋转，CNN在处理图像分类任务中仍有强大的能力。

#### 总体结论
- 自己实现的 kNN 实现优于 Scikit-learn 的实现，尤其在特定 $ k $ 值下表现突出。
- CNN 在图像旋转后仍有较高的分类精度，显示了深度学习在手写数字识别任务中的有效性。